## Machine Learning for Neuroscience, <br>Department of Brain Sciences, Faculty of Medicine, <br> Imperial College London
### Contributors: Antigone Fogel, Nan Fletcher-Lloyd, Anastasia Gailly De Taurines, Iona Biggart, Payam Barnaghi
Machine Learning for Neuroscience, <br>Department of Brain Sciences, Faculty of Medicine, <br> Imperial College London

**Spring 2026**

# Lab 4: Bayesian Models

This tutorial will focus on Bayesian theory/models using [scikit-learn](https://scikit-learn.org/stable).

In [ ]:
import sklearn as sk
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### **Likelihood and log-likelihood**

When we fit a probabilistic model, we often choose parameters that make the observed data **most likely**.

For a simple example, suppose we observe a sequence of binary outcomes (e.g., 1 = disease, 0 = no disease).
If the probability of disease is $\theta$, then the likelihood $P(\text{data} \mid \theta)$ tells us
how plausible different values of $\theta$ are given the data.

We often use the **log-likelihood** because it turns products into sums and is numerically more stable.

In the code cell below, we illustrate the idea of **likelihood** and **log-likelihood**
using a simple binary example. We assume the data are generated by a Bernoulli process
with an unknown probability of success $\theta$, and examine how plausible different
values of $\theta$ are given the observed data.

In [ ]:
data = np.array([1, 0, 1, 1, 0, 1, 1, 1, 0, 1])
n = len(data)
k = data.sum()

thetas = np.linspace(0.001, 0.999, 500)

log_likelihood = k * np.log(thetas) + (n - k) * np.log(1 - thetas)

theta_mle = k / n
print(f"MLE for theta = {theta_mle:.2f}")

plt.figure()
plt.plot(thetas, log_likelihood)
plt.axvline(theta_mle, linestyle="--")
plt.xlabel("theta")
plt.ylabel("log-likelihood (up to a constant)")
plt.title("Log-likelihood for a Bernoulli/Binomial model")
plt.show()

The curve shows how likely different values of $\theta$ are given the observed data.
The value of $\theta$ that maximises the log-likelihood is the **maximum likelihood
estimate (MLE)**. In practice, most machine learning libraries automatically perform
this optimisation when fitting probabilistic models, including Naive Bayes classifiers.

#### **Fitting a Bayesian Classifier**

Now, as you will have learnt in your lectures, **Naive Bayes** methods are a set of supervised learning classifiers that operate under the assumption that the presence of one feature in a class is independent of the presence of any other feature.

There are several types of Naive Bayes models (learn more [here](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes)) but for this exercise, we will focus on **[Gaussian Naive Bayes classifiers](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB)**.

For this, we will create our own dataset of **10 features** across **500 instances**, categorised into **two classes**: 
- **Class 0** represents a control group
- **Class 1** represents the class of interest. 
As in real world medical problems, there are fewer instances of the class of interest.

*Learn more about making your own classification datasets [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html)*

In [ ]:
features, labels = datasets.make_classification(n_samples=500,      # Total number of samples to generate
                                                n_features=10,      # Number of features in the dataset
                                                n_informative=8,    # Number of features that carry useful information
                                                n_classes=2,        # Number of classes
                                                weights=np.array([0.65, 0.35]), # Proportion of samples in each class
                                                flip_y=0.01,        # Fraction of samples to randomly flip (introduces noise)
                                                class_sep=2.0,      # Separability of classes - larger=more distinct
                                                shuffle=True,       # Shuffle dataset after generation
                                                random_state=42)

Now, let's take a quick look at the features and labels that were generated, in dataframe form for ease.

In [ ]:
features = pd.DataFrame(features)
labels = pd.DataFrame(labels)
df = pd.concat([features, labels], axis=1)
df.columns = ["F"+str(i) for i in range(10)] + ['Class']
df

For this data, we want to determine $P(\text{label} \mid \text{features})$ using scikit-learn's `predict_proba`/`predict` functions.

First, we must split our data into training and testing data.

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, shuffle=True, random_state = 42)

Next, we need to import and build our model.

In [ ]:
from sklearn.naive_bayes import GaussianNB
GNB = GaussianNB(var_smoothing=0.5)

In [ ]:
y_train = y_train.values.reshape(-1) # converts the DataFrame to a 1D array

GNB.fit(x_train, y_train) # fits the model on the training data
y_pred = GNB.predict(x_test) # predicts labels on the test data
y_pred_probs = GNB.predict_proba(x_test) # predicted probability of each class on the test data

Notice that we didn't scale our data before fitting the Gaussian Naive Bayes model. Is scaling necessary in this case? Why or why not?
> Answer here

Now, we want to evaluate the performance of the classification model.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [ ]:
f1 = f1_score(y_test, y_pred,average=None)
recall = recall_score(y_test, y_pred,average=None)
precision = precision_score(y_test, y_pred,average=None)


print('\nf1:\t\t',f1)
print('recall\t\t',recall)
print('precision\t',precision)

print('\nf1_avg:\t\t',f1.mean())
print('recall_avg\t',recall.mean())
print('precision_avg\t',precision.mean())

print('\nf1_sd:\t\t',f1.std())
print('recall_sd\t',recall.std())
print('precision_sd\t',precision.std())

print('\n',classification_report(y_test, y_pred))
print(roc_auc_score(y_test, y_pred_probs[:,1]))

#### **Probability calibration**

Some classifiers can output predicted probabilities, but these probabilities
are not always well calibrated. A well-calibrated model is one where predictions
of 0.7 correspond to events occurring about 70% of the time.

In the code cell below, we assess how well the predicted probabilities from a
Gaussian Naive Bayes classifier are calibrated.

In [ ]:
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss

y_proba = GNB.predict_proba(x_test)[:, 1] # probabilities for the positive class

# Compute calibration curve
fraction_of_positives, mean_predicted_value = calibration_curve(
    y_test,
    y_proba,
    n_bins=10
)

# Brier score (lower is better)
brier = brier_score_loss(y_test, y_proba)
print(f"Brier score: {brier:.3f}")

# Plot calibration curve
plt.figure()
plt.plot(mean_predicted_value, fraction_of_positives, marker="o", label="GaussianNB")
plt.plot([0, 1], [0, 1], linestyle="--", label="Perfect calibration")
plt.xlabel("Mean predicted probability")
plt.ylabel("Fraction of positives")
plt.title("Calibration curve")
plt.legend()
plt.show()

If the model is well calibrated, the curve should lie close to the diagonal.
Deviations from the diagonal indicate that predicted probabilities are either
too optimistic or too conservative.

The Brier score summarizes calibration quality, with lower values indicating
better calibrated predictions.

**CHALLENGE**: In the code cell below, train a Logistic Regression classifier on the same data. Plot a calibration curve for the LR model and compare it to the one we plotted above. Which model appears to be better calibrated?

In [ ]:
## CODE HERE ##


#### **ROC curve and AUC**

So far, we have evaluated our model using a fixed decision threshold (typically 0.5).
However, classifiers that output probabilities allow us to vary this threshold.

The Receiver Operating Characteristic (ROC) curve shows the trade-off between the
true positive rate (recall) and false positive rate across all possible thresholds.
The Area Under the Curve (AUC) summarizes this performance as a single value.

In [ ]:
from sklearn.metrics import roc_curve

y_proba = GNB.predict_proba(x_test)[:, 1] # probabilities for the positive class

fpr, tpr, thresholds = roc_curve(y_test, y_proba)

roc_auc = roc_auc_score(y_test, y_proba)

plt.figure()
plt.plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], linestyle="--", label="Chance")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate (Recall)")
plt.title("ROC Curve")
plt.legend()
plt.show()

**CHALLENGE**: In the code cell below, compute predictions/probabilities at thresholds of 0.5 and 0.2. How do the evaluation metrics change? Why might different thresholds be relevant in clinical settings?

In [ ]:
## CODE HERE ##


#### **Grid search**

Now, these scores are not very good, particularly when it comes to identifying the class of interest (Class 1). So how can we improve model performance?

One method is to **tune the hyperparameters** of the model (read more about this [here](https://medium.com/analytics-vidhya/how-to-improve-naive-bayes-9fa698e14cba))

For this, we first need to import the sklearn GridSearchCV function. This functions runs through all the different parameters fed into the parameter grid and produces the best combination of parameters based on a chosen scoring metric.

Learn more about how to use this function [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

In [ ]:
from sklearn.model_selection import GridSearchCV

Next, we need to set the range of all the parameters we will feed into the parameter grid. For a Gaussian Naive Bayes classifier, the only parameter to tune is `var_smoothing`, which represents a stability calculation to widen (or smooth) the curve and therefore account for more samples that are further away from the distribution mean.

In [ ]:
param_grid_nb = {'var_smoothing': np.logspace(0,-9, num=1000)} 

`np.logspace` returns numbers spaced evenly on a log scale, in the above example, we create a list of: 1000 samples starting from 0 and ending at -9 

Now, we build the GridSearchCV, using the model and parameter grid. We then fit this searching tool to the training data using a 10-fold cross-validation to find an optimal combination of hyperparameters that minimizes a predefined loss function to give better results.

In [ ]:
GNB_Grid = GridSearchCV(estimator=GaussianNB(var_smoothing=0.5), 
                        param_grid=param_grid_nb, 
                        verbose=1, 
                        cv=10, 
                        n_jobs=-1)
GNB_Grid.fit(x_train, y_train)

In [ ]:
print(GNB_Grid.best_estimator_)

Now we want to re-evaluate the model on the test data.

In [ ]:
grid_pred = GNB_Grid.predict(x_test)
grid_pred_probs = GNB_Grid.predict_proba(x_test) # predicted probability of each class on the test data

In [ ]:
f1 = f1_score(y_test, grid_pred,average=None)
recall = recall_score(y_test, grid_pred,average=None)
precision = precision_score(y_test, grid_pred,average=None)


print('\nf1:\t\t',f1)
print('recall\t\t',recall)
print('precision\t',precision)

print('\nf1_avg:\t\t',f1.mean())
print('recall_avg\t',recall.mean())
print('precision_avg\t',precision.mean())

print('\nf1_sd:\t\t',f1.std())
print('recall_sd\t',recall.std())
print('precision_sd\t',precision.std())

print('\n',classification_report(y_test, grid_pred))
print(roc_auc_score(y_test, grid_pred_probs[:,1]))

Now, these scores are much better! 

**CHALLENGE**: In the code cell below, plot the ROC AUC and calibration curves for your improved model. Bonus points if you can plot them in one multiplot.

In [ ]:
## CODE HERE ##


And there you have it! You've now built and tuned your first Bayesian classifer.

Now you've finished this tutorial, follow the instructions and complete the assessment.